In [67]:
import numpy as np

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup

def driversetup():
    options = webdriver.ChromeOptions()
    #run Selenium in headless mode
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    #overcome limited resource problems
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("lang=en")
    #open Browser in maximized mode
    options.add_argument("start-maximized")
    #disable infobars
    options.add_argument("disable-infobars")
    #disable extension
    options.add_argument("--disable-extensions")
    options.add_argument("--incognito")
    options.add_argument("--disable-blink-features=AutomationControlled")
    
    driver = webdriver.Chrome(options=options)

    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined});")

    return driver

In [2]:
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from bs4 import BeautifulSoup
def extract(url,date):
    #"https://www.wunderground.com/history/daily/VOMM/date/2023-3-14"
    try:
        driver = webdriver.Chrome()
        driver.get(url)
        tables = WebDriverWait(driver,20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "table")))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        table = soup.find('table',{'class':'mat-table cdk-table mat-sort ng-star-inserted'})
        time_list = []
        date_list = []
        temp_list = []
        hum_list = []

        for tag in table:
            td = tag.find_all('td', {'class': 'ng-star-inserted'})
            count = 0
            for t in  td:
                row=t.find_all('span',{'class': 'ng-star-inserted'})
                #print(row)
                flag = 0
                for r in row:
                    txt = r.text
                    if 'PM' in txt or 'AM' in txt:
                        time_list.append(txt)
                        count=1
                    elif count==1:
                        temp_list.append(txt)
                        count+=1
                    elif count==5:
                        hum_list.append(txt)
                        count+=1
                    else:
                        count+=1
        temp_df = pd.DataFrame({'Time':time_list,'Temp':temp_list,'Hum':hum_list})
        temp_df['Temp'] = temp_df['Temp'].astype('str').str.extractall('(\d+)').unstack().fillna('').sum(axis=1).astype(int)
        temp_df['Hum'] = temp_df['Hum'].astype('str').str.extractall('(\d+)').unstack().fillna('').sum(axis=1).astype(int)

        for i in range(0,len(temp_df['Time'])):
            if 'PM' in temp_df.loc[i,'Time']:
                val=temp_df.loc[i,'Time'].split(':')[0]
                if(val!='12'):
                    temp_df.loc[i,'Time']=str(int(val)+12)
                else:
                    temp_df.loc[i,'Time']=val
            elif 'AM' in temp_df.loc[i,'Time']:
                val=temp_df.loc[i,'Time'].split(':')[0]
                if(val!='12')and(int(val)<10):
                    temp_df.loc[i,'Time']='0'+val
                elif((val!='12')and(int(val)>=10)):
                    temp_df.loc[i,'Time']=val
                else:
                    temp_df.loc[i,'Time']='00'

        temp_df = temp_df.groupby(['Time']).mean().round(decimals=1).reset_index()
        L=list()
        L.append(date)
        temp_df['Date']=L*len(temp_df['Time'])
        #L=list(date)*len(temp_df['Time'])
        #temp_df['Date']=L
        driver.quit()
        return temp_df
    except:
        driver = webdriver.Chrome()
        driver.get(url)
        tables = WebDriverWait(driver,20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "table")))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        table = soup.find('table',{'class':'mat-table cdk-table mat-sort ng-star-inserted'})
        time_list = []
        date_list = []
        temp_list = []
        hum_list = []

        for tag in table:
            td = tag.find_all('td', {'class': 'ng-star-inserted'})
            count = 0
            for t in  td:
                row=t.find_all('span',{'class': 'ng-star-inserted'})
                #print(row)
                flag = 0
                for r in row:
                    txt = r.text
                    if 'PM' in txt or 'AM' in txt:
                        time_list.append(txt)
                        count=1
                    elif count==1:
                        temp_list.append(txt)
                        count+=1
                    elif count==5:
                        hum_list.append(txt)
                        count+=1
                    else:
                        count+=1
        temp_df = pd.DataFrame({'Time':time_list,'Temp':temp_list,'Hum':hum_list})
        temp_df['Temp'] = temp_df['Temp'].astype('str').str.extractall('(\d+)').unstack().fillna('').sum(axis=1).astype(int)
        temp_df['Hum'] = temp_df['Hum'].astype('str').str.extractall('(\d+)').unstack().fillna('').sum(axis=1).astype(int)

        for i in range(0,len(temp_df['Time'])):
            if 'PM' in temp_df.loc[i,'Time']:
                val=temp_df.loc[i,'Time'].split(':')[0]
                if(val!='12'):
                    temp_df.loc[i,'Time']=str(int(val)+12)
                else:
                    temp_df.loc[i,'Time']=val
            elif 'AM' in temp_df.loc[i,'Time']:
                val=temp_df.loc[i,'Time'].split(':')[0]
                if(val!='12')and(int(val)<10):
                    temp_df.loc[i,'Time']='0'+val
                elif((val!='12')and(int(val)>=10)):
                    temp_df.loc[i,'Time']=val
                else:
                    temp_df.loc[i,'Time']='00'

        temp_df = temp_df.groupby(['Time']).mean().round(decimals=1).reset_index()
        L=list()
        L.append(date)
        temp_df['Date']=L*len(temp_df['Time'])
        return temp_df

In [ ]:
df1=extract('https://www.wunderground.com/history/daily/VOMM/date/2023-3-14','2023-3-14')

In [ ]:
df1.head()

In [ ]:
df1.shape

In [ ]:
frames = [df1, df1]
df1 = pd.concat(frames)
df1.shape

In [3]:
from datetime import date, timedelta

start_dt = date(2021, 9, 1)
end_dt = date(2022, 8, 31)

# difference between current and previous date
delta = timedelta(days=1)

# store the dates between two dates in a list
dates = []

while start_dt <= end_dt:
    # add current date to list by converting  it to iso format
    dates.append(start_dt.isoformat())
    # increment start date by timedelta
    start_dt += delta

#dates

In [4]:
#
df_hr_wt=pd.DataFrame(columns=["Date","Time","Temperature","Humidity"])
for lnk in dates:
    url = 'https://www.wunderground.com/history/daily/VOMM/date/'+lnk
    df_new = extract(url,lnk)
    frames = [df_hr_wt, df_new]
    df_hr_wt = pd.concat(frames)

In [5]:
df_hr_wt.shape

(8727, 6)

In [6]:
df_hr_wt.head()

,Date,Time,Temperature,Humidity,Temp,Hum
0,2021-09-01,00,NaN,NaN,82.0,84.0
1,2021-09-01,01,NaN,NaN,82.0,84.0
2,2021-09-01,02,NaN,NaN,82.0,86.5
3,2021-09-01,03,NaN,NaN,82.0,89.0
4,2021-09-01,04,NaN,NaN,84.0,79.0


In [8]:
df_hr_wt.to_csv("Chennai_Weather.csv", sep=',', index=False)

In [74]:
df_hr_wt = pd.read_csv("Chennai_Weather.csv")

In [75]:
df_hr_wt.head()

,Date,Time,Temperature,Humidity,Temp,Hum
0,2021-09-01,0,NaN,NaN,82.0,84.0
1,2021-09-01,1,NaN,NaN,82.0,84.0
2,2021-09-01,2,NaN,NaN,82.0,86.5
3,2021-09-01,3,NaN,NaN,82.0,89.0
4,2021-09-01,4,NaN,NaN,84.0,79.0


In [76]:
df_hr_wt.shape

(8727, 6)

In [77]:
for i in dates:
    if i not in list(df_hr_wt['Date'].unique()):
        print(i)

In [78]:
df_hr_wt['Time']=df_hr_wt['Time'].astype(int)

In [79]:
ms_dates = []
for d in dates:
    ct=df_hr_wt.loc[df_hr_wt.Date == d, 'Time'].count()
    if (ct!=24):
        print(d,"   ",ct)
        ms_dates.append(d)

2021-09-29     23
2021-10-05     23
2021-11-01     23
2022-02-01     21
2022-05-18     23
2022-06-11     22
2022-07-02     23
2022-07-19     23
2022-08-18     13
2022-08-19     13


In [82]:
count=0
for i in ms_dates:
    for j in range(0,24):
        if j not in list(df_hr_wt.loc[df_hr_wt.Date == i, 'Time']):
            row = [i,j,np.nan,np.nan,np.nan,np.nan]
            #print(row)
            df_hr_wt.loc[len(df_hr_wt.index)] = row

In [83]:
df_hr_wt.shape

(8760, 6)

In [84]:
for d in dates:
    ct=df_hr_wt.loc[df_hr_wt.Date == d, 'Time'].count()
    if (ct!=24):
        print(d,"   ",ct)

In [98]:
df_hr_wt['Temp']=df_hr_wt['Temp'].fillna(df_hr_wt.groupby(df_hr_wt['Date'])['Temp'].transform('mean'))
df_hr_wt['Hum']=df_hr_wt['Hum'].fillna(df_hr_wt.groupby(df_hr_wt['Date'])['Hum'].transform('mean'))

In [99]:
df_hr_wt.isnull().sum()

Date              0
Time              0
Temperature    8760
Humidity       8760
Temp              0
Hum               0
dtype: int64

In [100]:
df_hr_wt = df_hr_wt.drop(['Temperature','Humidity'],axis=1)

In [101]:
df_hr_wt.head()

,Date,Time,Temp,Hum
0,2021-09-01,0,82.0,84.0
1,2021-09-01,1,82.0,84.0
2,2021-09-01,2,82.0,86.5
3,2021-09-01,3,82.0,89.0
4,2021-09-01,4,84.0,79.0


In [102]:
df_hr_wt.to_csv("Chennai_Weather.csv", sep=',', index=False)